<a href="https://colab.research.google.com/github/lx-47/gitaccess/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import userdata
github_token = userdata.get('github_token')
groq_api_key = userdata.get('groq_api_key')

In [1]:
!pip install groq pygithub python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 31.9 MB/s eta 0:00:00


In [11]:

# Step 2: Mount Google Drive (optional but recommended for saving results)
from google.colab import drive
drive.mount('/content/drive')

# Step 3: Set up authentication
from getpass import getpass
import os

# Get credentials

repo_url = "https://github.com/lx-47/1000iq-backend.git"

# Set environment variables
os.environ['GITHUB_TOKEN'] = github_token
os.environ['GROQ_API_KEY'] = groq_api_key

# Step 4: Clone the repository
!git clone {repo_url}
repo_name = repo_url.split('/')[-1].replace('.git', '')
repo_path = f"/content/{repo_name}"

# Step 5: Create code analyzer
from pathlib import Path
from groq import Groq

class ColabCodeAnalyzer:
    def __init__(self):
        self.groq = Groq(api_key=os.environ['GROQ_API_KEY'])
        self.model = "mixtral-8x7b-32768"  # Try "llama2-70b-4096" for different results

    def load_codebase(self, path: str) -> dict:
        """Load code from cloned repository"""
        codebase = {}
        for file in Path(path).rglob('*'):
            if file.is_file() and file.suffix in ('.py', '.md'):
                try:
                    codebase[str(file.relative_to(path))] = file.read_text()
                except UnicodeDecodeError:
                    continue
        return codebase

    def ask_question(self, codebase: dict, question: str):
        """Analyze codebase with Groq"""
        context = "\n".join([f"// File: {path}\n{content}"
                           for path, content in codebase.items()])

        prompt = f"""CODEBASE CONTEXT:
{context[:15000]}... [truncated]

TASK: {question}

INSTRUCTIONS:
1. Analyze existing architecture
2. Identify required files to modify/create
3. Generate the required code
"""

        response = self.groq.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model,
            temperature=0.3,
            max_tokens=8000
        )
        return response.choices[0].message.content


# Initialize analyzer
analyzer = ColabCodeAnalyzer()
codebase = analyzer.load_codebase(repo_path)

question = "a feature for students to chat with tutor"
print(analyzer.ask_question(codebase, question))

Mounted at /content/drive
Cloning into '1000iq-backend'...
remote: Enumerating objects: 404, done.
remote: Counting objects: 100% (404/404), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 404 (delta 256), reused 351 (delta 203), pack-reused 0 (from 0)
Receiving objects: 100% (404/404), 294.41 KiB | 5.45 MiB/s, done.
Resolving deltas: 100% (256/256), done.
1. Analyzing existing architecture:

Based on the provided codebase, the following components are relevant for implementing a chat feature for students to chat with tutors:

- Models: `User`, `Student`, `Tutor`, `Comment`
- Serializers: `UserSerializer`, `StudentSerializer`, `TutorSerializer`, `CommmentSerializer`
- Views: `StudentProfileView`, `TutorProfileView`

2. Identified files to modify/create:

- lms/models.py: Create a new model for chat messages.
- lms/serializers.py: Create a new serializer for chat messages.
- lms/views.py: Create new views for sending and receiving chat messages.

3. Generated code

In [3]:
!pip install -q groq pygithub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 31.0 MB/s eta 0:00:00


In [12]:
# Get credentials securely


from github import Github, InputGitTreeElement
import base64
from groq import Groq
import os

class RemoteCodeAnalyzer:
    def __init__(self):
        self.groq = Groq(api_key="sdf")
        self.gh = Github("dsf")
        self.repo = self.gh.get_repo("lx-47/1000iq-backend")
        self.model = "mixtral-8x7b-32768"  # Better model for code generation

    def get_codebase(self) -> dict:
        """Fetch code files including Python files"""
        contents = self.repo.get_contents("")
        return self._get_files_recursive(contents)

    def _get_files_recursive(self, contents) -> dict:
        """Include Python and notebook files"""
        codebase = {}
        for content in contents:
            if content.type == "dir":
                codebase.update(self._get_files_recursive(self.repo.get_contents(content.path)))
            elif content.path.endswith(('.py', '.md')):
                codebase[content.path] = base64.b64decode(content.content).decode()
                con = self.repo.get_contents(content.path)
        return codebase

    def generate_docstrings(self, codebase: dict) -> dict:
        """Generate docstrings for Python files using Groq AI."""

        # Select only Python files
        py_files = {path: content for path, content in codebase.items() if path.endswith(".py")}

        if not py_files:
            raise ValueError("❌ No Python files found in the repository.")

        context = "\n".join([f"# File: {path}\n{content}" for path, content in list(py_files.items())[:5]])
        print(context)
        prompt = f"""You are an expert Python Django developer. Add a chat feature for students and tutors.

        This is the existing code of the project: {context}
        Generate only the new required code with files names to which changes have to be made.
        """

        response = self.groq.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model,
            temperature=0.3,
            max_tokens=7000
        )
        return self._parse_response(response.choices[0].message.content)

    def _parse_response(self, response: str) -> dict:
        """Improved parsing with fallback"""
        files = {}
        current_file = None

        for line in response.split('\n'):
            if line.startswith('<') and line.endswith('>'):
                current_file = line[1:-1].strip()
                files[current_file] = []
            elif current_file:
                files[current_file].append(line)

        # Clean up code blocks
        return {
            path: '\n'.join([l for l in lines if not l.startswith('```')])
            for path, lines in files.items()
            if any(line.strip() for line in lines)
        }

    def create_pr(self, changes: dict):
        """Create a Pull Request with the updated files containing docstrings."""

        if not changes:
            raise ValueError("❌ No docstring updates detected. Cannot create an empty PR.")

        branch_name = "add-docstrings"

        # Ensure branch exists
        base_commit = self.repo.get_branch(self.repo.default_branch).commit
        self.repo.create_git_ref(
            ref=f"refs/heads/{branch_name}",
            sha=base_commit.sha
        )

        # Prepare file changes
        elements = []
        for path, content in changes.items():
            blob = self.repo.create_git_blob(content, "utf-8")
            elements.append(InputGitTreeElement(path=path, mode="100644", type="blob", sha=blob.sha))

        # Get base tree
        base_tree = self.repo.get_git_commit(base_commit.sha).tree

        # Create new tree
        tree = self.repo.create_git_tree(elements, base_tree)

        # Get parent commit correctly
        parent_commit = self.repo.get_git_commit(base_commit.sha)

        # Create commit
        commit = self.repo.create_git_commit(
            message="Added docstrings to Python files",
            tree=tree,
            parents=[parent_commit]  # ✅ FIXED
        )

        # Update branch reference
        branch_ref = self.repo.get_git_ref(f"heads/{branch_name}")
        branch_ref.edit(sha=commit.sha)

        # Create PR
        pr = self.repo.create_pull(
            title="Added docstrings to Python files",
            body="This PR adds PEP-257 compliant docstrings to all Python files.",
            head=branch_name,
            base=self.repo.default_branch
        )

        return pr.html_url

# Initialize analyzer
analyzer = RemoteCodeAnalyzer()

# Fetch codebase
codebase = analyzer.get_codebase()

# Generate docstrings for Python files
changes = analyzer.generate_docstrings(codebase)
print(changes)

# Create PR with updated docstrings
#pr_url = analyzer.create_pr(changes)

#print(f"✅ Pull Request Created: {pr_url}")


# File: backend/__init__.py

# File: backend/asgi.py
"""
ASGI config for backend project.

It exposes the ASGI callable as a module-level variable named ``application``.

For more information on this file, see
https://docs.djangoproject.com/en/5.1/howto/deployment/asgi/
"""

import os

from django.core.asgi import get_asgi_application

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'backend.settings')

application = get_asgi_application()

# File: backend/settings.py
"""
Django settings for backend project.

Generated by 'django-admin startproject' using Django 5.1.1.

For more information on this file, see
https://docs.djangoproject.com/en/5.1/topics/settings/

For the full list of settings and their values, see
https://docs.djangoproject.com/en/5.1/ref/settings/
"""

from pathlib import Path
from datetime import timedelta
# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(__file__).resolve().parent.parent

AUTH_USER_MODEL = 'lms.User'

# Quick-start dev

In [6]:
from github import Github, InputGitTreeElement
import base64
from groq import Groq
import os
import re

class RemoteCodeUpdater:
    def __init__(self):
        self.groq = Groq(api_key=groq_api_key)
        self.gh = Github(github_token)
        self.repo = self.gh.get_repo("lx-47/1000iq-backend")
        self.model = "mixtral-8x7b-32768"
        self.new_branch = "feature/chat"

    def get_codebase(self) -> dict:
        contents = self.repo.get_contents("")
        return self._get_files_recursive(contents)

    def _get_files_recursive(self, contents) -> dict:
        codebase = {}
        for content in contents:
            if content.type == "dir":
                codebase.update(self._get_files_recursive(self.repo.get_contents(content.path)))
            elif content.path.endswith(('.py', '.html', '.js')):
                codebase[content.path] = base64.b64decode(content.content).decode()
        return codebase

    def generate_feature_code(self) -> dict:
        prompt = """Generate Django chat feature code for these files:
        - chat/consumers.py (WebSocket)
        - chat/routing.py
        - chat/models.py

        Format:
        <chat/consumers.py>
        ```python
        from channels.generic.websocket import AsyncWebsocketConsumer
        class ChatConsumer(AsyncWebsocketConsumer):
            async def connect(self):
                await self.accept()
        ```
        </chat/consumers.py>"""

        response = self.groq.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model,
            temperature=0.3,
            max_tokens=6000
        )
        return self._parse_response(response.choices[0].message.content)

    def _parse_response(self, response: str) -> dict:
        files = {}
        current_file = None
        code_block = False

        for line in response.split('\n'):
            if re.match(r'^<([^>]+)>$', line):
                current_file = re.findall(r'^<([^>]+)>$', line)[0]
                files[current_file] = []
            elif line.strip() == '```':
                code_block = not code_block
            elif current_file and code_block:
                files[current_file].append(line)

        return {k: '\n'.join(v) for k, v in files.items() if v}

    def create_pr(self):
        changes = self.generate_feature_code()
        print("Detected changes:", changes.keys())

        # Create new branch
        self.repo.create_git_ref(
            ref=f"refs/heads/{self.new_branch}",
            sha=self.repo.get_branch("main").commit.sha
        )

        # Commit changes
        for path, content in changes.items():
            try:
                file = self.repo.get_contents(path)
                self.repo.update_file(
                    path=path,
                    message=f"Add chat feature to {path}",
                    content=content,
                    sha=file.sha,
                    branch=self.new_branch
                )
            except:
                self.repo.create_file(
                    path=path,
                    message=f"Create {path}",
                    content=content,
                    branch=self.new_branch
                )

        return self.repo.create_pull(
            title="Add Chat Feature",
            head=self.new_branch,
            base="main"
        ).html_url

# Usage
updater = RemoteCodeUpdater()
pr_url = updater.create_pr()
print(f"PR Created: {pr_url}")

Detected changes: dict_keys(['/chat/consumers.py', 'chat/routing.py', '/chat/models.py'])


GithubException: path cannot start with a slash: 422 {"message": "path cannot start with a slash", "errors": [{"resource": "Commit", "field": "path", "code": "invalid"}], "documentation_url": "https://docs.github.com/rest/repos/contents#create-or-update-file-contents", "status": "422"}

In [2]:
import os
import base64
from github import Github, InputGitTreeElement
from groq import Groq


class RemoteCodeUpdater:
    def __init__(self):
        """Initialize Groq AI and GitHub repository."""


        if not github_token or not groq_api_key:
            raise ValueError("Missing required environment variables: GITHUB_TOKEN and GROQ_API_KEY")

        self.groq = Groq(api_key=groq_api_key)
        self.gh = Github(github_token)
        self.repo = self.gh.get_repo(os.getenv("GITHUB_REPO", "lx-47/1000iq-backend"))
        self.model = os.getenv("GROQ_MODEL", "deepseek-r1-distill-llama-70b")
        self.new_branch = os.getenv("GIT_BRANCH", "feature/chat-feature")

    def get_repo_structure(self, path="") -> list:
        """Fetch the full directory structure of the repository."""
        contents = self.repo.get_contents(path)
        structure = []

        for content in contents:
            if content.type == "dir":
                structure.append(f"{content.path}/")
                structure.extend(self.get_repo_structure(content.path))
            else:
                structure.append(content.path)

        return structure

    def get_codebase(self) -> dict:
        """Fetch all files and their contents from the repo."""
        repo_structure = self.get_repo_structure()
        contents = self.repo.get_contents("")
        return self._get_files_recursive(contents)

    def _get_files_recursive(self, contents) -> dict:
        """Fetch Python, HTML, and other relevant files."""
        codebase = {}

        for content in contents:
            if content.type == "dir":
                codebase.update(self._get_files_recursive(self.repo.get_contents(content.path)))
            elif content.path.endswith(('.py', '.html', '.js', '.css', '.md')):
                try:
                    codebase[content.path] = base64.b64decode(content.content).decode("utf-8", errors="ignore")
                except Exception as e:
                    print(f"⚠️ Error decoding {content.path}: {e}")

        return codebase

    def generate_feature_code(self, codebase: dict) -> dict:
        """Generate feature modifications while ensuring correct file paths."""
        # Get repo structure for context
        repo_structure = "\n".join(self.get_repo_structure())

        # Prepare limited code samples for context
        context_files = []
        for path, content in list(codebase.items()):
            # Truncate very large files to avoid token limits
            if len(content) > 500:
                content = content[:100] + "\n# ... (file truncated for brevity)"
            context_files.append(f"# File: {path}\n{content}")

        context = "\n\n".join(context_files)

        prompt = f"""
        ### Repository Structure:
        {repo_structure}

        ### Code Samples:
        {context}

        ### Task: Add a Chat Feature
        Please modify the existing codebase to add a real-time chat feature using Django Channels:

        1. Only modify existing files when possible, and create new files only if necessary
        2. Follow Django best practices for WebSocket-based messaging
        3. Use correct file paths based on the repository structure
        4. Return complete file content for each modified file

        ### Response Format:
        <file_path>
        ```
        # File content here
        ```

        <another_file_path>
        ```
        # Another file content here
        ```
        """

        response = self.groq.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model,
            temperature=0.2,
            max_tokens=7000
        )

        return self._parse_response(response.choices[0].message.content)

    def _parse_response(self, response: str) -> dict:
        """Parse AI response into structured file modifications."""
        files = {}
        current_file = None
        current_content = []
        in_code_block = False

        lines = response.strip().split("\n")
        i = 0

        while i < len(lines):
            line = lines[i].strip()

            # Check for file path (not in code block and not empty)
            if not in_code_block and line and not line.startswith("```"):
                # If there's a current file being processed, save it
                if current_file and current_content:
                    files[current_file] = "\n".join(current_content).strip()
                    current_content = []

                # Found a new file path
                current_file = line

                # Skip ahead to find the opening code block
                i += 1
                while i < len(lines) and not lines[i].strip().startswith("```"):
                    i += 1

                # Start of code block
                if i < len(lines) and lines[i].strip().startswith("```"):
                    in_code_block = True
                    i += 1  # Skip the opening ```

            # End of code block
            elif in_code_block and line.startswith("```"):
                in_code_block = False

            # Inside code block - add to current content
            elif in_code_block:
                current_content.append(lines[i])

            i += 1

        # Save the last file
        if current_file and current_content:
            files[current_file] = "\n".join(current_content).strip()

        # Validate paths against actual repo structure
        valid_repo_files = set(self.get_repo_structure())
        validated_changes = {}

        for path, content in files.items():
            # Clean up the path (remove backticks, quotes, etc.)
            clean_path = path.strip('`"\' \t\n')

            if clean_path in valid_repo_files:
                # Path exists in repo
                validated_changes[clean_path] = content
            elif any(clean_path.endswith(part) for part in valid_repo_files):
                # Path might be relative, try to find matching file
                for valid_path in valid_repo_files:
                    if valid_path.endswith(clean_path):
                        validated_changes[valid_path] = content
                        break
            else:
                # New file - verify it has a valid parent directory
                parent_dir = os.path.dirname(clean_path)
                if parent_dir + "/" in valid_repo_files or parent_dir == "":
                    validated_changes[clean_path] = content

        return validated_changes

    def update_repo_files(self, changes: dict):
        """Update existing files and create new ones if needed in the repository."""
        if not changes:
            raise ValueError("❌ No valid file updates detected. Cannot create an empty PR.")

        # Get the default branch's latest commit
        base_commit = self.repo.get_branch(self.repo.default_branch).commit

        # Try to create a new branch, handle if it already exists
        try:
            self.repo.create_git_ref(ref=f"refs/heads/{self.new_branch}", sha=base_commit.sha)
            print(f"✅ Created new branch: {self.new_branch}")
        except Exception as e:
            print(f"⚠️ Note: {e}")
            print(f"ℹ️ Will update existing branch: {self.new_branch}")

        # Get existing file paths in the repo
        try:
            existing_files = {content.path for content in self.repo.get_contents("", ref=self.new_branch)}
        except:
            existing_files = {content.path for content in self.repo.get_contents("")}

        # Process each file change
        for path, updated_content in changes.items():
            try:
                if path in existing_files:
                    # Update existing file
                    file = self.repo.get_contents(path, ref=self.new_branch)
                    self.repo.update_file(
                        path=path,
                        message=f"Update {path} with chat feature",
                        content=updated_content,
                        sha=file.sha,
                        branch=self.new_branch
                    )
                    print(f"✅ Updated existing file: {path}")
                else:
                    # Create new file
                    self.repo.create_file(
                        path=path,
                        message=f"Create {path} for chat feature",
                        content=updated_content,
                        branch=self.new_branch
                    )
                    print(f"✅ Created new file: {path}")
            except Exception as e:
                print(f"❌ Error with file {path}: {e}")

    def create_pull_request(self):
        """Create a Pull Request for the changes."""
        try:
            pr = self.repo.create_pull(
                title="Add Chat Feature",
                body="This PR adds a real-time chat feature using Django Channels.",
                head=self.new_branch,
                base=self.repo.default_branch
            )
            print(f"✅ Created PR: {pr.html_url}")
            return pr.html_url
        except Exception as e:
            print(f"❌ Error creating PR: {e}")
            return None




if __name__ == "__main__":

    # Run the updater
    updater = RemoteCodeUpdater()
    codebase = updater.get_codebase()
    changes = updater.generate_feature_code(codebase)

    print(f"Generated changes for {len(changes)} files:")
    for path in changes.keys():
        print(f"- {path}")
        print(changes[path])

    proceed = input("\nDo you want to apply these changes to the repository? (y/n): ")
    if proceed.lower() == 'y':
        updater.update_repo_files(changes)

        create_pr = input("Create a pull request? (y/n): ")
        if create_pr.lower() == 'y':
            pr_url = updater.create_pull_request()
            if pr_url:
                print(f"Pull request created: {pr_url}")
    else:
        print("Operation cancelled.")

Generated changes for 3 files:
- <think>
Django settings for backend project.

Generated by 'django-admin startproject' using Django 5.1.
"""

from pathlib import Path

# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(__file__).resolve().parent.parent

# SECURITY WARNING: keep the str of your secret key used in production secret!
SECRET_KEY = 'your-secret-key-here'

# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = True

ALLOWED_HOSTS = []

# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'lms',
    'chat',  # Add chat app
    'channels',
    'rest_framework',
]

MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django

In [30]:
print(codebase)

{'README.md': '"# backend" \n', 'backend/__init__.py': '', 'backend/asgi.py': '"""\nASGI config for backend project.\n\nIt exposes the ASGI callable as a module-level variable named ``application``.\n\nFor more information on this file, see\nhttps://docs.djangoproject.com/en/5.1/howto/deployment/asgi/\n"""\n\nimport os\n\nfrom django.core.asgi import get_asgi_application\n\nos.environ.setdefault(\'DJANGO_SETTINGS_MODULE\', \'backend.settings\')\n\napplication = get_asgi_application()\n', 'backend/settings.py': '"""\nDjango settings for backend project.\n\nGenerated by \'django-admin startproject\' using Django 5.1.1.\n\nFor more information on this file, see\nhttps://docs.djangoproject.com/en/5.1/topics/settings/\n\nFor the full list of settings and their values, see\nhttps://docs.djangoproject.com/en/5.1/ref/settings/\n"""\n\nfrom pathlib import Path\nfrom datetime import timedelta\n# Build paths inside the project like this: BASE_DIR / \'subdir\'.\nBASE_DIR = Path(__file__).resolve()

In [20]:
# ✅ Update the repository with changes
updater.update_existing_files(changes)

# ✅ Create a PR for merging
pr_url = updater.create_pull_request()
print(f"🚀 Pull Request Created: {pr_url}")

⚠️ Skipping invalid file: backend/chat/consumers.py
⚠️ Skipping invalid file: backend/settings.py
⚠️ Skipping invalid file: backend/urls.py
⚠️ Skipping invalid file: backend/chat/templates/chat.html
⚠️ Skipping invalid file: backend/lms/views.py
⚠️ Skipping invalid file: backend/lms/urls.py


GithubException: Validation Failed: 422 {"message": "Validation Failed", "errors": [{"resource": "PullRequest", "code": "custom", "message": "No commits between main and feature/chat-feature"}], "documentation_url": "https://docs.github.com/rest/pulls/pulls#create-a-pull-request", "status": "422"}

In [1]:
!pip install pygithub Groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 27.4 MB/s eta 0:00:00


In [19]:
from github import Github
import os
gh = Github(github_token)
repo = gh.get_repo(os.getenv("GITHUB_REPO", "lx-47/1000iq-backend"))
def list_all_files_in_a_dir(folder_name):
  a = repo.get_contents(folder_name,ref = "main")
  for i in a :
    print(i.path)

content=""
target_files = {"models.py", "views.py"}

def read_all_files_in_a_dir(folder_name):
  global content
  for i in repo.get_contents(folder_name):
    if(i.type == "dir"):
      print(i.path)
      read_all_files_in_a_dir(i.path)
    if i.type == "file" and i.path.endswith(".py"):
        file_name = i.path.split("/")[-1]
        if file_name in target_files:
                file_content = i.decoded_content.decode()
                content += f"### {i.path}\n\n{file_content}\n\n"

list_all_files_in_a_dir("lms")
read_all_files_in_a_dir("lms")
print(content)

lms/__init__.py
lms/__pycache__
lms/admin.py
lms/apps.py
lms/migrations
lms/models.py
lms/serializers.py
lms/signals.py
lms/tests.py
lms/urls.py
lms/views.py
lms/__pycache__
lms/migrations
lms/migrations/__pycache__
### lms/models.py

from email.mime import image
from itertools import count
from django.db import models
from django.contrib.auth.models import AbstractUser
from django.conf import settings

class User(AbstractUser):
    Choices=[
        ('student','Student'),
        ('tutor','Tutor'),
        ('admin','Admin'),
    ]
    role = models.CharField(max_length=8, choices=Choices)

    def __str__(self):
        return self.username

class Student(models.Model):
    image = models.CharField(max_length=255,default="")
    banner = models.CharField(max_length=255,default="") 
    first_name = models.CharField(max_length = 50, null = True, blank = True)
    last_name = models.CharField(max_length = 50, null = True, blank=True)        
    user = models.OneToOneField(settings.AUTH

In [35]:
from github import Github
import ast
import os
from google.colab import userdata
github_token = userdata.get('github_token')
# Authenticate with GitHub
gh = Github(github_token)

# Get the repository
repo = gh.get_repo(os.getenv("GITHUB_REPO", "lx-47/1000iq-backend"))

# Function to list all Python files recursively
def get_all_python_files(folder_name, ref="main"):
    files = []
    contents = repo.get_contents(folder_name, ref=ref)

    for content in contents:
        if content.type == "dir":
            files.extend(get_all_python_files(content.path))
        elif content.type == "file" and content.path.endswith(".py"):
            files.append(content.path)

    return files


In [36]:
import ast
def parse_python_file(file_path):
    file_content = repo.get_contents(file_path).decoded_content.decode("utf-8")

    try:
        # Parse content into an AST
        tree = ast.parse(file_content)
        return tree
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return None


In [37]:
# Extract symbols from AST
def extract_symbols(tree, file_path):
    symbols = {"functions": [], "classes": [], "variables": []}

    # Walk through the AST nodes
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            symbols["functions"].append({"name": node.name, "line": node.lineno, "file": file_path})

        elif isinstance(node, ast.ClassDef):
            symbols["classes"].append({"name": node.name, "line": node.lineno, "file": file_path})

        elif isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name):
                    symbols["variables"].append({"name": target.id, "line": node.lineno, "file": file_path})

    return symbols


In [39]:
def create_symbol_index(folder_name):
    symbol_index = {"functions": [], "classes": [], "variables": []}

    # Get all Python files in the directory
    python_files = get_all_python_files(folder_name)

    for file_path in python_files:
        tree = parse_python_file(file_path)

        if tree:
            symbols = extract_symbols(tree, file_path)

            symbol_index["functions"].extend(symbols["functions"])
            symbol_index["classes"].extend(symbols["classes"])
            symbol_index["variables"].extend(symbols["variables"])

    return symbol_index

# Generate the symbol index for the 'lms' directory
symbol_index = create_symbol_index("lms")
print(symbol_index)
for i in symbol_index["functions"]:
  print(i)


{'functions': [{'name': 'ready', 'line': 8, 'file': 'lms/apps.py'}, {'name': '__str__', 'line': 15, 'file': 'lms/models.py'}, {'name': '__str__', 'line': 27, 'file': 'lms/models.py'}, {'name': '__str__', 'line': 47, 'file': 'lms/models.py'}, {'name': 'get_course_count', 'line': 50, 'file': 'lms/models.py'}, {'name': 'get_specializations', 'line': 53, 'file': 'lms/models.py'}, {'name': 'get_average_rating', 'line': 56, 'file': 'lms/models.py'}, {'name': '__str__', 'line': 96, 'file': 'lms/models.py'}, {'name': 'get_student_count', 'line': 99, 'file': 'lms/models.py'}, {'name': 'get_student_ids', 'line': 102, 'file': 'lms/models.py'}, {'name': 'get_section_count', 'line': 106, 'file': 'lms/models.py'}, {'name': 'total_duration', 'line': 109, 'file': 'lms/models.py'}, {'name': 'get_lesson_count', 'line': 113, 'file': 'lms/models.py'}, {'name': 'get_average_rating', 'line': 116, 'file': 'lms/models.py'}, {'name': '__str__', 'line': 150, 'file': 'lms/models.py'}, {'name': 'total_duration', 

In [29]:
from groq import Groq

def call_model(query):
  client = Groq(
      api_key=groq_api_key,
  )

  prompt = f'''
  You are an expert django developer who wants to generate production level code for the {query} provided as input.
  -You are tasked with reading all backend files in the lms directory {content}
  -Generate the views.py code only between <views></views>
  '''

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="mistral-saba-24b",
      temperature=0.1,
      max_tokens=6000
  )
  return chat_completion.choices[0].message.content

new_content = call_model("Add a new feature for students and tutors to chat.")

In [32]:
import re
views_match = re.search(r"<views>(.*?)</views>", new_content, re.DOTALL)
views_content = views_match.group(1).strip()
print(views_content)

from rest_framework.generics import RetrieveUpdateAPIView, UpdateAPIView
from django.shortcuts import get_object_or_404
from rest_framework.views import APIView
from rest_framework.viewsets import ModelViewSet, ReadOnlyModelViewSet
from lms.models import (
    Assessment,
    Comment,
    Course,
    CourseEnrollment,
    CourseRating,
    Lesson,
    Question,
    Reward,
    Section,
    Student,
    StudentAssessment,
    Todo,
    Tutor,
    User,
    Message  # Import the new Message model
)
from lms.serializers import (
    AssessmentSerializer,
    ChangePasswordSerializer,
    CommmentSerializer,
    CourseEnrollmentSerializer,
    CourseRatingSerializer,
    CourseSerializer,
    CourseSerializer2,
    LessonSerializer,
    QuestionSerializer,
    RewardSerializer,
    SectionSerializer,
    StudentAssessmentSerializer,
    StudentSerializer,
    TodoSerializer,
    TutorSerializer,
    UserSerializer,
    MessageSerializer  # Import the new MessageSerializer
)
from rest_frame

In [30]:
def create_branch(branch_name):
  source_branch = repo.get_branch(repo.default_branch)
  source_sha = source_branch.commit.sha

  repo.create_git_ref(ref=f"refs/heads/{branch_name}", sha=source_sha)

  print(f"Branch '{branch_name}' created based on '{repo.default_branch}'")

create_branch("new-feature")


Branch 'new-feature' created based on 'main'


In [34]:
from github import InputGitTreeElement
def new_commit(branch,file_path,new_content):
  base_ref = repo.get_git_ref(f"heads/{branch}")
  base_sha = base_ref.object.sha
  base_tree = repo.get_git_commit(base_sha).tree

  element = InputGitTreeElement(
      path=file_path,
      mode='100644',
      type='blob',
      content=new_content)

  # Create a tree
  tree = repo.create_git_tree([element], base_tree)

  parent = repo.get_git_commit(base_sha)
  commit = repo.create_git_commit("commit message", tree, [parent])

  base_ref.edit(commit.sha)

  print(f"Changes committed to branch '{branch}'")

new_commit("new-feature","lms/models.py",views_content)


Changes committed to branch 'new-feature'


In [26]:
import requests
headers = {"Authorization":github_token}
response = requests.get("https://api.github.com/repos/lx-47/1000iq-backend/contents/lms/models.py", headers=headers)
issues = response.json()
print(issues)

<Response [200]>
{'name': 'models.py', 'path': 'lms/models.py', 'sha': '2f7271b61b8169036682012349f5a2fd81a95867', 'size': 9971, 'url': 'https://api.github.com/repos/lx-47/1000iq-backend/contents/lms/models.py?ref=main', 'html_url': 'https://github.com/lx-47/1000iq-backend/blob/main/lms/models.py', 'git_url': 'https://api.github.com/repos/lx-47/1000iq-backend/git/blobs/2f7271b61b8169036682012349f5a2fd81a95867', 'download_url': 'https://raw.githubusercontent.com/lx-47/1000iq-backend/main/lms/models.py', 'type': 'file', 'content': 'ZnJvbSBlbWFpbC5taW1lIGltcG9ydCBpbWFnZQpmcm9tIGl0ZXJ0b29scyBp\nbXBvcnQgY291bnQKZnJvbSBkamFuZ28uZGIgaW1wb3J0IG1vZGVscwpmcm9t\nIGRqYW5nby5jb250cmliLmF1dGgubW9kZWxzIGltcG9ydCBBYnN0cmFjdFVz\nZXIKZnJvbSBkamFuZ28uY29uZiBpbXBvcnQgc2V0dGluZ3MKCmNsYXNzIFVz\nZXIoQWJzdHJhY3RVc2VyKToKICAgIENob2ljZXM9WwogICAgICAgICgnc3R1\nZGVudCcsJ1N0dWRlbnQnKSwKICAgICAgICAoJ3R1dG9yJywnVHV0b3InKSwK\nICAgICAgICAoJ2FkbWluJywnQWRtaW4nKSwKICAgIF0KICAgIHJvbGUgPSBt\nb2RlbHMuQ2hhckZpZWxkKG1heF9sZW